### 소비자 행동데이터 분석을 위한 2019년 10월 1개월 간의 소비자 행동데이터

In [1]:
# #Elasticsearch 설치
# !pip install elasticsearch

In [2]:
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

In [3]:
# 데이터 읽기
file_path = r"C:\Users\SEC\Downloads\2019-Oct.csv"
data = pd.read_csv(file_path)

In [4]:
# 데이터프레임의 크기 확인
print("데이터프레임의 크기:", data.shape)

데이터프레임의 크기: (42448764, 9)


In [5]:
data.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [6]:
# # 큰 크기의 CSV 파일을 더 작은 청크(chunk)로 나누어 읽고 이를 데이터프레임에 추가
# chunks = pd.read_csv(file_path, encoding='utf-8', chunksize=1000)

In [7]:
# # Elasticsearch 서버 연결
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

In [8]:
# Elasticsearch 서버에 ping 요청 보내기
if es.ping():
    print("Elasticsearch 서버에 성공적으로 연결되었습니다.")
else:
    print("Elasticsearch 서버에 연결하지 못했습니다.")

Elasticsearch 서버에 성공적으로 연결되었습니다.


In [9]:
# Elasticsearch에 매핑 설정
oct_index = 'oct_index'  # 원하는 색인명으로 변경

mapping = {
    "mappings": {
        "properties": {
            "brand": {"type": "text"},
            "category_code": {"type": "text"},  # 'category_code' 필드를 문자열로 지정
            # 다른 필드들에 대한 매핑 추가
        }
    }
}

In [15]:
# 매핑을 설정한 인덱스에 적용
es.indices.create(index=oct_index, body=mapping, ignore=400)

C:\Users\SEC\AppData\Local\Temp\ipykernel_20436\3122518104.py:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index=oct_index, body=mapping, ignore=400)


ObjectApiResponse({'error': {'root_cause': [{'type': 'resource_already_exists_exception', 'reason': 'index [oct_index/1DuBzKScQuiW5p20MlsRow] already exists', 'index_uuid': '1DuBzKScQuiW5p20MlsRow', 'index': 'oct_index'}], 'type': 'resource_already_exists_exception', 'reason': 'index [oct_index/1DuBzKScQuiW5p20MlsRow] already exists', 'index_uuid': '1DuBzKScQuiW5p20MlsRow', 'index': 'oct_index'}, 'status': 400})

In [11]:
# # 큰 크기의 CSV 파일을 더 작은 청크(chunk)로 나누어 읽고 이를 데이터프레임에 추가
# chunks = pd.read_csv(file_path, encoding='utf-8', chunksize=1000)

In [12]:
# Elasticsearch에 이미 색인된 데이터 삭제
es.indices.delete(index=oct_index, ignore=[400, 404])

C:\Users\SEC\AppData\Local\Temp\ipykernel_20436\4103419027.py:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index=oct_index, ignore=[400, 404])


ObjectApiResponse({'acknowledged': True})

In [13]:
# 큰 크기의 CSV 파일을 작은 청크(chunk)로 나누어 읽고 이를 Elasticsearch에 색인
chunk_size = 1000
for chunk in pd.read_csv(file_path, encoding='utf-8', chunksize=chunk_size):
    # 각 청크에 대한 처리를 여기에 추가
    # 예를 들어, 원하는 계산 수행 또는 데이터 전처리 등을 수행

    # NaN을 0으로 대체
    chunk = chunk.fillna(0)

    # Elasticsearch에 색인할 때 사용할 데이터프레임 생성
    for i, row in chunk.iterrows():
        document = row.to_dict()

        # 'brand'와 'category_code' 필드를 명시적으로 문자열로 변환
        document['brand'] = str(document['brand'])
        document['category_code'] = str(document['category_code'])

        try:
            # Elasticsearch에 색인
            es.index(index=oct_index, body=document)
        except Exception as e:
            print(f"Error indexing document: {e}")
            print(f"Error details: {document}")

print("색인이 완료되었습니다.")

Error indexing document: Connection timed out
Error details: {'event_time': '2019-10-01 08:53:12 UTC', 'event_type': 'view', 'product_id': 3800490, 'category_id': 2053013566176363511, 'category_code': 'appliances.iron', 'brand': 'maxwell', 'price': 15.19, 'user_id': 515279577, 'user_session': '241e5063-736d-445a-be8f-f40da01ddf77'}
Error indexing document: Connection timed out
Error details: {'event_time': '2019-10-01 10:42:59 UTC', 'event_type': 'view', 'product_id': 5300147, 'category_id': 2053013563173241677, 'category_code': '0', 'brand': 'rowenta', 'price': 46.31, 'user_id': 525944235, 'user_session': '30fe53ab-e69e-4642-b04f-da2e58e136a5'}
Error indexing document: Connection timed out
Error details: {'event_time': '2019-10-01 12:40:57 UTC', 'event_type': 'view', 'product_id': 17300355, 'category_id': 2053013553853497655, 'category_code': '0', 'brand': 'creed', 'price': 234.24, 'user_id': 521484178, 'user_session': 'f6fcc9a7-45fc-468d-9aaf-df31af69f48c'}
Error indexing document: C

KeyboardInterrupt: 

In [14]:
import elasticsearch
print(elasticsearch.__version__)

(8, 11, 1)
